Running requires having the data on Google Drive or uploading the data to Colab via the left menu and setting the data path correctly.

To use TPU, set "tpu" to True and hardware accelerator to "TPU" from Edit -> Notebook Settings

In [ ]:
seed = 42
import pandas as pd
import numpy as np
import random
np.random.seed(seed)
random.seed(seed)
tpu = False
from google.colab import drive
drive.mount('/gdrive')
data_path = '/gdrive/My Drive/Colab Notebooks/'

In [ ]:
import os
import tensorflow as tf
# This address identifies the TPU we'll use when configuring TensorFlow.
if tpu:
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  tf.logging.set_verbosity(tf.logging.INFO)

def tpu_compatibilitate(model):
  if tpu: return tf.contrib.tpu.keras_to_tpu_model(
      model, strategy=tf.contrib.tpu.TPUDistributionStrategy(
              tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))
  else: return model

Read data

In [ ]:
data = pd.read_csv(data_path + 'EEG_data.csv')
data.columns

The labels for confusion are the same for each video by subject

In [ ]:
for subjId in set(data.SubjectID):
  for vidId in set(data.VideoID):
    assert data.query('SubjectID == {} and VideoID == {}'
                     .format(subjId, vidId))['user-definedlabeln'].mean() in (0.0, 1.0)

Load subtitle vectors  


In [ ]:

import numpy as np
"""
from csv
"""
#vid_dfs = pd.concat([pd.read_csv(notebook_path + 'subtitles/vid_{}_elmo_embedded_subs.csv'.format(i))
#                     for i in range(10)], ignore_index=True
#                   ).sort_values(['SubjectID', 'VideoID']).reset_index(drop=True)

#vec_cols = [str(x) for x in range(1024)]

#sub_vecs = vid_dfs[vec_cols].values.astype('float32')

"""
save/load from npy
"""
sub_vec_path = data_path + 'subtitle_vecs.npy'
#np.save(sub_vec_path, sub_vecs)
sub_vecs = np.load(sub_vec_path)
sub_vec_dim = sub_vecs.shape[1]

"""
Make a dataset of original data combined with sub vecs 
"""
dataset = np.hstack((data.values.astype('float32'), sub_vecs))

PCA to reduce subtitle vector dimensions. Speeds up training and also has the potential to increase performance

In [ ]:
from sklearn.decomposition import PCA
"""
PCA to reduce dimension of the word average vectors (might give better results)
"""
sub_vec_dim = 12
pca = PCA(n_components=sub_vec_dim)
pcad_sub_vecs = pca.fit_transform(sub_vecs)
#dataset = np.hstack((data.values.astype('float32'), pcad_sub_vecs))

In [ ]:
dataset.shape

Preprocessing as is done in https://github.com/mehmani/DNNs-for-EEG-Signals/blob/master/DNNforEEFSignals.ipynb

In [ ]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler

def NormSignal(S, I):
    #normalize features
    S=S.reshape(-1, 1)
    if I not in [0, 1, 13, 14]:
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(S)
        scaled = scaled
    else:
        scaled = S
    return scaled.reshape(-1).tolist()

NormDataG = np.array([NormSignal(dataset[:,i], i) for i in range(dataset.shape[1])]).T
print(NormDataG.shape)

Additional metrics besides accuracy to have more information on model performance 

In [ ]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def swap_tensor_binary_ints(x):
    return tf.math.add(tf.negative(x), 1.)

with tf.Session() as sess:
  print("Test swapping tensor binary ints")
  diagonal = tf.linalg.diag(tf.ones((1, 2)))
  print('orig:\n', diagonal.eval())
  print('swap:\n', swap_tensor_binary_ints(diagonal).eval())

def f1_score(y_true, y_pred):
    y_true =y_true
    y_pred = tf.round(y_pred) # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 2 * precision * recall / (precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)
  
def f1_flipped(y_true, y_pred):
    y_true = y_true
    y_pred = tf.round(y_pred) # implicit 0.5 threshold via tf.round
    return f1_score(swap_tensor_binary_ints(y_true), 
                    swap_tensor_binary_ints(y_pred))

def rocauc(y_true, y_pred):
    roc_auc_score(y_true, y_pred)

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Flatten, Dropout, TimeDistributed, Conv2D, MaxPooling2D, Masking

"""
Model from the paper "confused or not confused"
"""
def get_model_timedist(intervals, n_dim=11):
    model = Sequential([
        #Masking(mask_value=0, input_shape=(A, n_dim)), # Masking does not help for some reason (should help with padded data?)
        BatchNormalization(input_shape=(intervals, n_dim), axis=2), # New version of keras doesn't support "mode" attribute, which was used in the original code (mode=0)
        Bidirectional(LSTM(50, return_sequences=False, activation='selu'), input_shape=(intervals, n_dim)),
        Dense(intervals, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='RMSprop',
                  metrics=['binary_accuracy', f1_score, f1_flipped])
    return model, (-1, intervals, n_dim)


In [ ]:
"""
Model from the paper "confused or not confused" with binary output per data point

"""
def get_model(intervals, n_dim=11):
    model = Sequential([
        #Masking(mask_value=0, input_shape=(A, n_dim)), # Masking does not help for some reason (should help with padded data?)
        BatchNormalization(input_shape=(intervals, n_dim), axis=2),
        Bidirectional(LSTM(50, return_sequences=False, activation='selu'), input_shape=(intervals, n_dim)),
        #Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adagrad',
                  metrics=['acc', f1_score, f1_flipped])
    return model, (-1, intervals, n_dim)

In [ ]:
def get_mehmani_model(intervals, n_dim=11):
    model = Sequential()
    model.add(TimeDistributed(Conv2D(20, (5,5), activation='relu'),
              input_shape=(1, intervals, n_dims, 1)))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(10, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(20, return_sequences=True)))
    model.add(LSTM(10))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', f1_score, f1_flipped])
    return model, (-1, 1, intervals, n_dim, 1)

Functions for making the data amount of intervals the same for each data point

In [ ]:
def min_max_rows_per_subject_vid(X):
  VideoID = list(set(X[:,1]))
  SubjectID = list(set(X[:,0]))

  max_intervals = 0 # length of signal
  min_intervals = len(X)
  
  for subId in SubjectID:
      for vidId in VideoID:
          X_tmp=X[(X[:, 0] == subId) & (X[:, 1] == vidId)]
          max_intervals = max(len(X_tmp), max_intervals)
          min_intervals = min(len(X_tmp), min_intervals)
  print(max_intervals)
  print(min_intervals)
  assert max_intervals == 144
  return min_intervals, max_intervals

min_intervals, max_intervals = min_max_rows_per_subject_vid(dataset)


def zero_pad_data(X, max_intervals, y_col):
  # Manual Padding to fixed size:
    X_pad = None
    VideoID = list(set(X[:,1]))
    SubjectID = list(set(X[:,0])) 
    for subId in SubjectID:
        for vidId in VideoID:
            X_sv = X[(X[:,0]==subId) & (X[:,1]==vidId)]
            pad_len = max_intervals - X_sv.shape[0]
            
            z = np.zeros((pad_len, X_sv.shape[1]), dtype=X_sv.dtype)
            z[:,0] = X_sv[:,0][pad_len]
            z[:,1] = X_sv[:,1][pad_len]
            z[:,y_col] = X_sv[:,y_col][pad_len]
            
            X_sv_pad = np.concatenate((X_sv, z), axis=0)
            X_sv_pad = X_sv_pad.reshape(1, max_intervals, -1)

            X_pad = X_sv_pad if X_pad is None else np.vstack((X_pad,X_sv_pad))
            
    return X_pad

def truncate_data(X, min_intervals, y_col):
    X_trunc = None
    VideoID = list(set(X[:,1]))
    SubjectID = list(set(X[:,0]))
    for vidId in VideoID:
      for subId in SubjectID:
          X_sv = X[(X[:,0]==subId) & (X[:,1]==vidId)]
          trunc_len = min_intervals
          X_sv_trunc = X_sv[0:trunc_len].reshape(1, min_intervals, -1)
          X_trunc = X_sv_trunc if X_trunc is None else np.vstack((X_trunc, X_sv_trunc))
    return X_trunc




Define target variable and which variables to use for training

In [ ]:

y_col = 14 # The student's confusion column
orig_train_data_cols = list(range(2,14))
vector_cols = list(np.arange(sub_vec_dim) + 15) 

train_cols = orig_train_data_cols + vector_cols
n_dim = len(train_cols)

Cross-validation

In [ ]:

from time import time

def train_eval_model(model, X_train, y_train, X_test, y_test, train_cols, intervals,
                     epochs=20, batch_size=20, verbose=1): 
    start = time()

    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_test, y_test), verbose=verbose, shuffle=True)
    print('model trained in {:.3f} seconds'.format(time() - start))

    loss, acc, f1, f1_flip = model.evaluate(X_test, y_test, verbose=0)
    #y_pred = model.predict(X_test).ravel()
    #roc_auc = rocauc(y_test, y_pred)
    return (acc, f1, f1_flip, history)

 
def cross_validate(model, X_shape, data, intervals, even_data, n_test=2,
                   time_distributed=False, verbose=1, epochs=50, batch_size=20):
  """
  even_data: either truncate_data or zero_pad_data to make number of intervals
             even for each data point
  """
  results = []
  initial_weights = model.get_weights()

  for i in range(0, 10, n_test):
    model.set_weights(initial_weights) # Reset weights to forget training done on current iteration's test data  
    
    data_train = even_data(data[np.in1d(data[:,0], (i, i+1), invert=True)], intervals, y_col=y_col)
    data_test = even_data(data[np.in1d(data[:,0], (i, i+1))], intervals, y_col=y_col)
    X_train = data_train[:, :, train_cols]
    y_train = data_train[:, :, y_col]
    X_test = data_test[:, :, train_cols]
    y_test = data_test[:, :, y_col]

    X_train = X_train.reshape(X_shape)
    print('Xtrain shape', X_train.shape)
    X_test = X_test.reshape(X_shape)
    print('Xtest shape', X_test.shape)
    
    if not time_distributed: y_train = y_train.reshape(-1, intervals).mean(axis=1)
    print('ytrain shape', y_train.shape)
    if not time_distributed: y_test = y_test.reshape(-1, intervals).mean(axis=1)
    print('ytest shape', y_test.shape)

    start = time()
    print('{}-fold cross validation, iteration {}'
          .format(int(10/n_test), len(results) +1))
    acc, f1, f1_flip, history = train_eval_model(model, X_train, y_train,
                                                 X_test, y_test,
                                                 train_cols, intervals,
                                                 epochs=epochs, batch_size=batch_size,
                                                 verbose=verbose)
    
    results.append({'acc': acc, 'F1': f1, 'F1-flipped': f1_flip})
    
    print('current cross-validation mean accuracy: {:.3f}, f1: {:.3f}, and f1 flipped: {:.3f}'.format(
          *[np.mean([r[key] for r in results]) for key in results[0].keys()]))
    print('cross-validation total time: {:.3f} seconds'.format(time() - start))

  return results



In [ ]:
"""
Test Mehmani's model 
"""
#model, input_shape = get_mehmani_model(max_intervals, n_dims)
#results = cross_validate(model=model, X_shape=input_shape, data=NormDataG,
#                         intervals=max_intervals, even_data=zero_pad_data, n_test=2)

In [ ]:
#print(*results, sep='\n')
#print('cross-validation mean accuracy: {:.3f}, f1: {:.3f}, and f1 flipped: {:.3f}'.format(
#       *[np.mean([r[key] for r in results]) for key in results[0].keys()]))

In [ ]:
"""
Test the model from the paper "Confused or not confused"
"""
train_cols = orig_train_data_cols
n_dim = len(train_cols)
model, input_shape = get_model_timedist(min_intervals, n_dim)
tpu_model = tpu_compatibilitate(model)

results = cross_validate(model=tpu_model, X_shape=input_shape, data=dataset,
                         time_distributed=True,
                         intervals=min_intervals, even_data=truncate_data,
                         n_test=2, verbose=1, epochs=25, batch_size=20)

In [ ]:
print(*results, sep='\n')
print('cross-validation mean accuracy: {:.3f}, f1: {:.3f}, and f1 flipped: {:.3f}'.format(
       *[np.mean([r[key] for r in results]) for key in results[0].keys()]))

In [ ]:
"""
Test the model from the paper "Confused or not confused" with sub vectiors
"""
train_cols = orig_train_data_cols + vector_cols
n_dim = len(train_cols)
model, input_shape = get_model_timedist(min_intervals, n_dim)
tpu_model = tpu_compatibilitate(model)

results = cross_validate(model=tpu_model, X_shape=input_shape, data=dataset,
                         time_distributed=True,
                         intervals=min_intervals, even_data=truncate_data,
                         n_test=2, verbose=1, epochs=25, batch_size=20)

In [ ]:
"""
Test mehmani model with subtitle vectors
"""
print(NormDataG.shape)
train_cols = vector_cols
n_dims = len(train_cols)
model, input_shape = get_mehmani_model(max_intervals, n_dims)

tpu_model = tpu_compatibilitate(model)
results = cross_validate(model=tpu_model, X_shape=input_shape, data=NormDataG,
                         intervals=max_intervals, even_data=zero_pad_data, n_test=2)


In [ ]:
print(*results, sep='\n')
print('cross-validation mean accuracy: {:.3f}, f1: {:.3f}, and f1 flipped: {:.3f}'.format(
       *[np.mean([r[key] for r in results]) for key in results[0].keys()]))